以 H 门为例, 在 S-T0 单比特模型下训练参数实现 H 门。

基于 Mindquantum 0.10 和 MindSpore 2.2

In [16]:

import mindspore as ms
import mindspore.numpy as np
from mindquantum import (
    Circuit,
    H,
    Hamiltonian,
    MQN2Layer,
    PRGenerator,
    QubitOperator,
    Rn,
    Simulator,
    random_state,
)
from mindquantum.algorithm.library.amplitude_encoder import amp_circuit
from mindspore.nn import Adam, LossBase, ReLU, TrainOneStepCell, WithLossCell
from mindspore.ops import square

ms.set_seed(1)
ms.set_context(mode=ms.PYNATIVE_MODE, device_target="CPU")

params = PRGenerator() # 自动生成参数

random_circ = amp_circuit(n_qubits=1) # 以振幅编码线路作为随机线路
random_circ.as_encoder()              # 随机线路不参与训练

def ST0_circ(num: int=1, dt=np.pi/2):
    '''
    ST0 原生门序列组成拟设线路。
    
    Args:
        num: 原生门数量。
        dt: 原生门的脉冲宽度
    
    Return:
        Circuit.
    '''
    circ = Circuit()
    for i in range(num):
        circ += Rn(dt, 0, params.new()*dt).on(0) # Rn 的三个参数分别为哈密顿量 Sx, Sy, Sz 的系数。
    return circ

st0_circ = ST0_circ(12) # 搭建原生门组成的拟设线路
target_gate = H.on(0)   # 以 H 门作为目标门

circ_right = random_circ + st0_circ   # 实得线路
circ_left = random_circ + target_gate # 目标线路

ham = Hamiltonian(QubitOperator(''))  # 哈密顿量为 I
sim = Simulator('mqvector', 1)        # 模拟器
grad_ops = sim.get_expectation_with_grad(ham,
                                         circ_left,
                                         circ_right)

class MyN2layer(MQN2Layer):
    '''量子层可求最终结果的模方。自定义以实现脉冲强度的非负约束。'''
    def __init__(self, expectation_with_grad, weight='ones'): # ones 初始化保证初始参数非负
        super().__init__(expectation_with_grad, weight)
        self.relu = ReLU() # ReLu 保证参数处于非负区间
        
    def construct(self, arg):
        return self.evolution(arg, self.relu(self.weight)) # 对参数要施加 ReLu 操作
    
qlayer = MyN2layer(grad_ops)
    
class FidelityLoss(LossBase):
    '''根据保真度计算损失函数。自定义以去掉 label 参数'''
    def construct(self, logits):
        x = square(logits - 1)    # 保真度 F 区间为 [0, 1], F == 1 时损失函数达到最小值 0
        return self.get_loss(x)

f_loss = FidelityLoss()
    
class MyWithLossCell(WithLossCell):
    '''将量子层与损失函数拼接起来。自定义以去掉 label 参数'''
    def __init__(self, backbone, loss_fn):
        super().__init__(backbone, loss_fn)

    def construct(self, data): # 不需要 label 参数
        out = self._backbone(data)
        return self._loss_fn(out)
    
model = MyWithLossCell(qlayer, f_loss) # 组合量子层和损失函数
opti = Adam(model.trainable_params(), learning_rate=0.05) # 优化器配置

train = TrainOneStepCell(model, opti) # 最终组合

## 训练
steps = 201 # 训练步数
random_data = np.randn((steps, 1, len(random_circ.params_name))) # 预先生成随机线路的参数
for i in range(steps):
    res = train(random_data[i]) # 对随机线路的参数随机赋值，以生成随机量子态
    if i % 20 == 0:
        print(i, ': ', res)
        
params = abs(qlayer.weight.asnumpy()) # 最终参数，注意要加 abs 操作，以满足物理约束
print('\n最终的参数为:', params)

## 验证
random_states = random_state((4,2), 1)

fid = 1.
for state in random_states:
    target = target_gate.matrix() @ state
    actual = st0_circ.matrix(params) @ state
    tem = abs(sum(target.conj()*actual))**2
    fid = min(fid, tem)

print('\n保真度为:', fid)
print('\n误差为:', "{:e}".format(1-fid))


0 :  0.07866901
20 :  0.0064255907
40 :  2.2979593e-05
60 :  5.036319e-06
80 :  1.4111553e-08
100 :  3.042479e-06
120 :  1.4136555e-07
140 :  2.1611615e-06
160 :  8.3018236e-08
180 :  6.7991675e-07
200 :  8.8611375e-07

最终的参数为: [1.0634943 1.1799479 1.0904506 1.1092283 1.1675032 1.0943042 1.0522159
 1.1537404 1.1169224 1.0867579 1.1057397 1.1633892]

保真度为: 0.9990488369055909

误差为: 9.511631e-04
